# Multi-Layer Perceptron
In this Jupyter Notebook, we code a Multi-Layer Perceptron (MLP) for classification and regression tasks using Stochastic Gradient Descent for training the model. The MLP would be able to perform multiclass as well as binary classification. <br>

The model would also allow customization of activation function used, as well as training schedule for optimization purposes. The MLP is designed with a single hidden layer architecture. <br>

This notebook is dvided into three sections
1. Section 1 - Helper functions and multiclass classification
1. Section 2 - Binary classification
1. Section 3 - Regression

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import itertools
import math
from math import exp
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

## Section 1

### 1. Implement the following function that creates a weight matrix and initializes it with small random real numbers.

In [4]:
def initializeWeights(input_neurons,output_neurons):
    W = np.random.randn(output_neurons,input_neurons) * 0.01
    return W

### 2. Implement the logistic sigmoid activation function.

In [5]:
def logistic(z):
    return 1/(1+np.exp(-z))

### 3. Implement the ReLU (rectified linear unit) activation function.

In [6]:
def relu(z):
    return np.maximum(0,z) 

### 4. Implement the tanh (hyperbolic tangent) activation function.

In [7]:
def tanh(z):
    return np.tanh(z)

### 5. Implement a MLP Classifier model class for performing multi-class classification.

### The MLP Classifier has a single hidden layer. It should have the following four methods.The model uses the back-propagation algorithm for learning the weights of the features/neurons. Note the that “fit” method should implement the Stochastic Gradient Descent algorithm for optimizing the weight update process.

In [8]:
'''
Helper functions for the derivative of activation functions
'''
def d_logistic(z):
    dz = logistic(z) * (1 - logistic(z))
    return dz

def d_tanh(z):
    dz = (1-np.power(z,2))
    return dz

def d_relu(z):
    dz = np.where(z==0,0,1)
    return dz
    

In [9]:
'''
Helper function for one_hot_labels
Input:
    Y - target vector
Output:
    one_hot_matrix
'''
def one_hot_labels(Y):
    
    # Get unique labels in Y and number of observations
    unique_labels = np.unique(Y)
    n = len(Y)
    
    # Create one hot matrix
    labels = np.array(list(unique_labels)).reshape(len(unique_labels),1)
    one_hot_matrix = np.apply_along_axis(lambda x: np.full((n,),x),1,labels)
    one_hot_matrix = np.apply_along_axis(lambda x: (x==Y).astype(int),1,one_hot_matrix).T
    
    return one_hot_matrix

In [10]:
'''
Helper function for softmax calculation
Given matrix A (output from hidden layer), outputs probabilties   
'''
def softmax(A):
    A = np.apply_along_axis(np.exp,0,A)
    A = np.apply_along_axis(lambda x: x/(sum(x)),1,A)
    return A

In [11]:
'''
Helper function that determines the number of input layers
and the number of layers
Input:
    X - feature matrix
    Y - one_hot_matrix
    n_h - number of hidden layers
Output:
    n_o - tuple containing the values
'''
def size_of_layers(X,Y,n_h):
    n_i = X.shape[1]
    n_o = Y.shape[1]
    
    return n_i, n_h, n_o

In [16]:
'''
Helper function to initialize parameters of the MLP
This assumes a single hidden layer architecture
Bias is included
Input:
    n_i - size of input layer (# of X_train columns)
    n_h - size of hidden layer neurons
    n_o - size of output layer neurons (# number of classes)
Output:
    parameters - Dictionary of Parameters
'''
def initializeParameters(n_i, n_h, n_o):
    # Weights of each layer
    W_i = initializeWeights(n_i,n_h)
    W_h = initializeWeights(n_h,n_o)
    
    # Bias of each layer
    b_i = np.zeros((n_h,1))
    b_h = np.zeros((n_o,1))
    
    parameters = {
        'W_input': W_i,
        'b_input': b_i,
        'W_hidden': W_h,
        'b_hidden': b_h
    }
    
    return parameters

In [17]:
Y = one_hot_labels(y)

In [18]:
n_i, n_h, n_o = size_of_layers(X,Y,4)
params = initializeParameters(n_i, n_h, n_o)

In [19]:
'''
Forward propagation function given parameters, feature matrix and
activation function and function to calculate probabilities
Returns value after passing through hidden layer A2 and caching for backprop

Input:
    parameters - parameters
    X - feature matrix
    activation_function
    classifier - sigmoid or softmax for output layer
Output:
    A_h - output of hidden layer
    cache
'''
def forward_propagation(parameters, X, activation_function, classifier=softmax):
    # Parameters definition
    W_i = parameters['W_input']
    b_i = parameters['b_input']
    W_h = parameters['W_hidden']
    b_h = parameters['b_hidden']
    
    # Computation of Input Layer
    Z_i = np.dot(W_i,X.T) + b_i
    A_i = activation_function(Z_i)
    
    # Computation of Hidden Layer
    Z_h = np.dot(W_h,A_i) + b_h
    A_h = classifier(Z_h.T)
    
    # Caching 
    cache = {
        'Z_i': Z_i,
        'A_i': A_i,
        'Z_h': Z_h,
        'A_h': A_h
    }
    
    return A_h, cache
    

In [21]:
'''
Helper function to calculate the cross entropy cost function
Input:
    pred - predicted values
    Y - actual values
Output:
    cost
'''
def cost(pred, Y):
    cost = np.multiply(Y, np.log(pred)) + np.multiply((1-Y), np.log(pred))
    cost = -np.sum(cost)/Y.shape[0]
    return cost

In [22]:
'''
Helper function to select a sample with correct shape from X (feature matrix)
Input:
    X
Output:
    i - index
    x - vector of sample
'''
def select_sample(X):
    
    m,n = X.shape
    i = random.randint(0,m)
    
    return i, X[i].reshape(1,n)

In [23]:
'''
Calculation of backward propagation and passing down 
gradients.
Input:
    parameters 
    cache 
    X - can be vector or matrix
    Y - dependent of type of X
    d_activation - derivative of the activation fx used
Output
    grad - dictionary of gradients
'''
def back_propagation(parameters, cache, X, Y, d_activation):

    # Parameters of model
    W_i = parameters['W_input']
    b_i = parameters['b_input']
    W_h = parameters['W_hidden']
    b_h = parameters['b_hidden']

    # Retrieve Cached Calculations
    A_i = cache['A_i']
    A_h = cache['A_h']
    Z_i = cache['Z_i']
    Z_h = cache['Z_h']

    # Calculation of gradients 
    dz_h = A_h - Y
    dw_h = np.dot(A_i,dz_h).T
    db_h = np.sum(dz_h)
    dz_i = np.dot(dz_h,W_h) * d_activation(A_i).T
    dw_i = np.dot(dz_i.T,s)
    db_i = np.sum(dz_i)
    
    # Gradient Descent
    grad = {
        'dw_h' : dw_h,
        'db_h' : db_h,
        'dw_i' : dw_i,
        'db_i' : db_i
    }
    
    return grad


In [26]:
def gradient_descent(parameters, gradient, learning_rate):
    # Parameters of model
    W_i = parameters['W_input']
    b_i = parameters['b_input']
    W_h = parameters['W_hidden']
    b_h = parameters['b_hidden']
    
    # Gradient to update
    dw_h = gradient['dw_h']
    db_h = gradient['db_h']
    dw_i = gradient['dw_i']
    db_i = gradient['db_i']
    
    # Update
    W_i = W_i - learning_rate*dw_i
    b_i = b_i - learning_rate*db_i
    W_h = W_h - learning_rate*dw_h
    b_h = b_h - learning_rate*db_h
    
    parameters = {
        'W_input': W_i,
        'b_input': b_i,
        'W_hidden': W_h,
        'b_hidden': b_h
    }
    
    return parameters

### TESTING GROUND

In [38]:
# Convert into one-hot labels for Y
Y = one_hot_labels(y)

# Obtain size of layers setting 4 hidden neurons
n_i, n_h, n_o = size_of_layers(X,Y,4)
# Initialize parameters
params = initializeParameters(n_i, n_h, n_o)

# Get number of samples included and randomly select a value
i,s = select_sample(X)

A_h, cache = forward_propagation(params, s, relu, classifier=softmax)

grad = back_propagation(params, cache, s, Y[i], d_relu)

# Parameters of model
W_i = params['W_input']
b_i = params['b_input']
W_h = params['W_hidden']
b_h = params['b_hidden']

# Retrieve Cached Calculations
A_i = cache['A_i']
A_h = cache['A_h']
Z_i = cache['Z_i']
Z_h = cache['Z_h']

In [41]:
epochs = 100
learning_rate = 0.001

training_cost = []

for e in range(epochs):
    # select sample for SGD
    i,s = select_sample(X)

    # Run forward prop
    A_h, cache = forward_propagation(params, s, relu, classifier=softmax)
    
    training_cost.append(cost(A_h,Y))
    
    # obtain backprop from gradient descent
    grad = back_propagation(params, cache, s, Y[i], d_relu)
    
    params =  gradient_descent(params, grad, learning_rate)
    

In [42]:
training_cost

[23.090769228079992,
 23.035626361955412,
 23.1553869836875,
 23.032289289386622,
 23.032955817992672,
 23.036734272356245,
 24.602968856342276,
 23.047171771747262,
 23.13237215252103,
 23.029637816809633,
 23.10301597393142,
 23.025875524611333,
 23.03512361518619,
 24.085104826893343,
 23.080500332950667,
 24.253238838640534,
 23.068830380091708,
 23.346516214848933,
 23.222723645535595,
 23.02617355455827,
 23.27960359166309,
 23.495060589043295,
 23.05165503595681,
 23.15099510066276,
 23.07256930762801,
 23.048500521067425,
 23.040444044306856,
 23.062168507973055,
 23.027217428879982,
 23.034835840614832,
 23.056642866640203,
 23.02778553828076,
 23.09671689506662,
 23.0368890679229,
 23.035577773201133,
 23.028449866779173,
 23.10411244083284,
 23.070717607208326,
 23.041748580324857,
 23.030923880985114,
 23.11797553455881,
 23.02588254507033,
 23.214793383436444,
 24.130153485644037,
 23.027274931265282,
 23.13615384531266,
 23.027873735389527,
 23.508815272559694,
 24.112709

In [228]:
# Calculation of gradients 
dz_h = A_h - Y[i]
dw_h = np.dot(A_i,dz_h).T
db_h = np.sum(dz_h)
dz_i = np.dot(dz_h,W_h) * d_relu(A_i).T
dw_i = np.dot(dz_i.T,s)
db_i = np.sum(dz_i)

In [37]:
W

NameError: name 'W' is not defined

In [232]:
dw_h.shape

(10, 4)

In [233]:
W_h.shape

(10, 4)

In [234]:
dw_i.shape

(4, 64)

In [235]:
W_i.shape

(4, 64)

### 6. Read the handwritten digits datasetusing the sklearn.datasets.load_digits function for performing multi-class classification.

In [31]:
digits = datasets.load_digits()
X = digits.data
y = digits.target

In [32]:
X.shape

(1797, 64)

### 7. Partition the data into train and test set. Use the “Partition” function from your previous assignment or from sklearn.

In [33]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

### 8. Standardize the features.

In [34]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### 9. Hyperparameter tuning based on certain fixed-values hyperparameters.

### 10. Report on performance of model.

## Section 2

### 11. Implement binary classification module in theMLPClassifier. Then, performbinary classification on the handwritten digits dataset to recognize the digits “5” and “not-5”.

### (i) Hyperparameter tuning based on certain fixed-values hyperparameters

### (ii) Report performance on model.

## Section 3 (Extra Credit)

### 12. Implement a Multi-Layer Perceptron regressor model (a MLP Regressor class) with a single hidden layer. The model implements the backpropagation algorithm. To optimize the process of updating the weight matrices, it uses the Stochastic Gradient Descent (SGD) algorithm with momentum.

### (i) Hyperparameter tuning based on certain fixed-values hyperparameters

### (ii) Report performance on model.